In [1]:
import time

import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import seaborn as sns
from skimage.transform import resize

In [2]:
train_images = np.load("Kannada_MNIST_npz/Kannada_MNIST/X_kannada_MNIST_train.npz")['arr_0']
test_images = np.load("Kannada_MNIST_npz/Kannada_MNIST/X_kannada_MNIST_test.npz")['arr_0']
train_labels = np.load("Kannada_MNIST_npz/Kannada_MNIST/y_kannada_MNIST_train.npz")['arr_0']
test_labels = np.load("Kannada_MNIST_npz/Kannada_MNIST/y_kannada_MNIST_test.npz")['arr_0']

In [3]:
def pad3D(c_x, padlen=1):
    m,n,r = c_x.shape
    c_y = np.zeros((m, n+2*padlen, r+2*padlen),dtype=c_x.dtype)
    c_y[:, padlen:-padlen, padlen:-padlen] = c_x
    return c_y

In [4]:
#train_reshape = resize(train_images, (60000, 48, 48))
train_images = pad3D(train_images, padlen=2)
train_images.shape
test_images = pad3D(test_images, padlen=2)
test_images.shape

(10000, 32, 32)

In [5]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [6]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
img_rows, img_cols = 32, 32
batch_size = 128
num_classes = 10
epochs = 15

In [8]:
x_train = np.expand_dims(train_images, axis=-1)
x_test = np.expand_dims(test_images, axis=-1)
y_train = keras.utils.to_categorical(train_labels)
y_test = keras.utils.to_categorical(test_labels)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=71)

In [10]:
model = VGG16(weights=None, include_top=True, input_shape=(32, 32, 1), pooling = 'avg', classes = 10)
#model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', 
              metrics=['acc',f1_m,precision_m, recall_m])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          #callbacks=[mcp_save],
          validation_data=(x_valid, y_valid))




Train on 59400 samples, validate on 600 samples
Epoch 1/15
59400/59400 [==============================] - 136s 2ms/step - loss: 2.2165 - acc: 0.1596 - f1_m: 0.0466 - precision_m: 0.1051 - recall_m: 0.0335 - val_loss: 1.1497 - val_acc: 0.4750 - val_f1_m: 0.4588 - val_precision_m: 0.7045 - val_recall_m: 0.3417
Epoch 2/15
59400/59400 [==============================] - 138s 2ms/step - loss: 0.1726 - acc: 0.9491 - f1_m: 0.9482 - precision_m: 0.9573 - recall_m: 0.9407 - val_loss: 0.0977 - val_acc: 0.9833 - val_f1_m: 0.9833 - val_precision_m: 0.9850 - val_recall_m: 0.9817
Epoch 3/15
59400/59400 [==============================] - 140s 2ms/step - loss: 0.0560 - acc: 0.9873 - f1_m: 0.9874 - precision_m: 0.9893 - recall_m: 0.9856 - val_loss: 0.0403 - val_acc: 0.9933 - val_f1_m: 0.9917 - val_precision_m: 0.9933 - val_recall_m: 0.9900
Epoch 4/15
59400/59400 [==============================] - 140s 2ms/step - loss: 0.0837 - acc: 0.9814 - f1_m: 0.9816 - precision_m: 0.9847 - recall_m: 0.9785 - val_

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(shear_range = 0.5)
test_generator = datagen.flow(x_test, y_test, batch_size=batch_size)
test_nb_samples = len(y_test)

In [ ]:
images = np.load("Kannada_MNIST_npz/Dig_MNIST/X_dig_MNIST.npz")['arr_0']
labels = np.load("Kannada_MNIST_npz/Dig_MNIST/y_dig_MNIST.npz")['arr_0']
images_resized = pad3D(images, padlen=2)
x_digi_test = np.expand_dims(images_resized, axis=-1)
y_digi_test = keras.utils.to_categorical(labels)
datagen = ImageDataGenerator(shear_range = 0.5)
digi_generator = datagen.flow(x_digi_test, y_digi_test, batch_size=batch_size)
digi_nb_samples = len(y_digi_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
def plot_confusion_matrix(confusion_matrix, y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix#confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes#[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
loss, acc, f1, precision, recall = model.evaluate_generator(test_generator,steps = len(test_generator), verbose=0)
print("acc: ", acc,"\n f1: ",f1,"\nprecion: ", precision, "\nrecall:", recall)

In [ ]:
loss, acc, f1, precision, recall = model.evaluate_generator(digi_generator,steps = len(test_generator), verbose=0)
print("acc: ", acc,"\n f1: ",f1,"\nprecision: ", precision, "\nrecall:", recall)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

pred = model.predict(x_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(y_test, axis=1), predicted, digits = 4)
confusion_matrix = confusion_matrix(y_true=np.argmax(y_test, axis=1), y_pred=predicted)
print(report)
plot_confusion_matrix(confusion_matrix,y_true=np.argmax(y_test, axis=1), y_pred=predicted, classes=['0','1','2','3','4','5','6','7','8','9'],
                      title='Confusion matrix, without normalization')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

pred = model.predict(x_digi_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(y_digi_test, axis=1), predicted, digits = 4)
confusion_matrix = confusion_matrix(y_true=np.argmax(y_digi_test, axis=1), y_pred=np.argmax(pred, axis=1))
print(report)
plot_confusion_matrix(confusion_matrix,y_true=np.argmax(y_digi_test, axis=1), y_pred=predicted, classes=['0','1','2','3','4','5','6','7','8','9'],
                      title='Confusion matrix, without normalization')

In [ ]:
#plot_cm(np.argmax(y_digi_test, axis=1), np.argmax(pred, axis=1))

In [ ]:
print(model.summary())

In [ ]:
from sklearn.metrics import confusion_matrix

pred = model.predict(x_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
confusion_matrix = confusion_matrix(y_true=np.argmax(y_test, axis=1), y_pred=predicted)

In [ ]:
print(confusion_matrix)

In [ ]:
print(np.argmax(y_test, axis=1))